# DIRAC Analysis of LC M004 Proteomics with LC M001 Proteomics — Statistical Tests (GOBP Modules)

***by Kengo Watanabe***  

In the main Python notebook, the differential rank conservation (DIRAC; Eddy, J.A. et al. PLoS Comput. Biol. 2010) analysis is performed on the preprocessed Longevity Consortium (LC) M001 and M004 proteomics datasets (analytes detected in all samples; sample-based robust Z-score followed by analyte-based robust Z-score) using the retrieved a priori module set (Gene Ontology (Biological Process) derived by EMBL-EBI QuickGO API; ≥4 analytes and ≥50% coverage).  
**–> To maintain the consistency with the other DIRAC analyses, statistical tests are performed in this sub-notebook with R kernel.**  

Input:  
* Cleaned module metadata: 220529_LCproteomics-M004-DIRAC_DIRAC-GOBP-with-M001_ver2-3_module-metadata.tsv  
* Combined sample–mouse metadata: 220529_LCproteomics-M004-DIRAC_DIRAC-GOBP-with-M001_ver2-3_sample-metadata.tsv  
* Combined table of DIRAC RMSs: 210128_LCproteomics-M004-DIRAC_M001-M004-common-DIRAC-GOBP_QuickGO-GOBP_min-n4-cov50_RankMatchingScore-BS.tsv  
* Combined table of DIRAC RCIs: 210128_LCproteomics-M004-DIRAC_M001-M004-common-DIRAC-GOBP_QuickGO-GOBP_min-n4-cov50_RankConservationIndex-BS.tsv  

Output:  
* Supplementary Data 7  

Original notebook (memo for my future tracing):  
* dalek:[JupyterLab HOME]/210125_LCproteomics-M004-DIRAC/220529_LCproteomics-M004-DIRAC_StatisticalTest-GOBP-with-M001_ver2-3.ipynb  

In [ ]:
library("tidyverse")
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

#CRAN
for (package in c("multcomp", "openxlsx")) {
    #install.packages(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

## 1. Prepare dataset and metadata

In [ ]:
#Import module metadata
fileDir <- "./ExportData/"
ipynbName <- "220529_LCproteomics-M004-DIRAC_DIRAC-GOBP-with-M001_ver2-3_"
fileName <- "module-metadata.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")
print(str_c("nrow: ",nrow(temp)))
head(temp)

module_meta <- temp

In [ ]:
#Import sample-mouse metadata
fileDir <- "./ExportData/"
ipynbName <- "220529_LCproteomics-M004-DIRAC_DIRAC-GOBP-with-M001_ver2-3_"
fileName <- "sample-metadata.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")
print(str_c("nrow: ",nrow(temp)))
head(temp)

sample_meta <- temp

## 2. Perform DIRAC with sex-pooled rank consensus

In [ ]:
#Import the DIRAC results
fileDir <- "./ExportData/"
ipynbName <- "210128_LCproteomics-M004-DIRAC_M001-M004-common-DIRAC-GOBP_"
fileName <- "QuickGO-GOBP_min-n4-cov50_RankMatchingScore-BS.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t") %>%
    dplyr::rename(ModuleID=NetworkID)
print(str_c("nrow: ",nrow(temp)))
head(temp)
rms_tbl <- temp

fileName <- "QuickGO-GOBP_min-n4-cov50_RankConservationIndex-BS.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t") %>%
    dplyr::rename(ModuleID=NetworkID)
print(str_c("nrow: ",nrow(temp)))
head(temp)
rci_tbl <- temp

## 3. Rank conservation index: general pattern

### 3-1. Extract RCI (the mean of RMSs under the own phenotype consensus)

In [ ]:
#Extract RCI whose template phenotype corresponds to the own phenotype
phenotype_vec <- rci_tbl %>%
    dplyr::select(-ModuleID, -Template) %>%
    names()
temp <- tibble(ModuleID=unique(rci_tbl$ModuleID))
for (k in phenotype_vec) {
    temp <- rci_tbl %>%
        dplyr::filter(Template==!!k) %>%
        dplyr::select(ModuleID, !!k) %>%
        dplyr::left_join(temp, ., by="ModuleID")
}
#Order and re-label
group_vec <- c("M001:Cont", "M001:Acar", "M001:17aE", "M001:Rapa", "M004:Cont", "M004:4EGI")
temp <- temp %>%
    dplyr::rename(`M001:Acar`=Acarbose, `M001:Cont`=`Control (M001)`,
                  `M001:17aE`=Estradiol, `M001:Rapa`=Rapamycin,
                  `M004:Cont`=`Control (M004)`, `M004:4EGI`=`4EGI-1`) %>%
    dplyr::select(ModuleID, all_of(group_vec))
print(str_c("nrow: ",nrow(temp)))
head(temp)
summary(temp)

rci_kk <- temp

### 3-2. Repeated Student's t-tests

> In this study, RCI was not normalized (i.e., the expected mean and variance were different between M001 and M004 datasets due to different sample size).  
> –> Therefore, not Dunnett's test but Student's t-test (i.e., t-test with pooled variance) is used because RCI is NOT comparable across datasets (i.e., Control group is different between datasets).  

#### 3-2-1. Simultaneously perform all tests

In [ ]:
#Prepare contrast labels
contrast_vec <- c("M001:Acar-vs-M001:Cont", "M001:17aE-vs-M001:Cont", "M001:Rapa-vs-M001:Cont",
                  "M004:4EGI-vs-M004:Cont")

#Prepare DF
temp1 <- rci_kk %>%
    tidyr::gather(key=Group, value=RCI, -ModuleID)
temp <- tibble()
for (comparison in contrast_vec) {
    contrast <- str_split(comparison, "-vs-", simplify=TRUE)[1]
    baseline <- str_split(comparison, "-vs-", simplify=TRUE)[2]
    temp2 <- temp1 %>%
        dplyr::filter(Group==!!contrast) %>%
        dplyr::select(ModuleID, Group, RCI) %>%
        dplyr::rename(Group_contrast=Group,
                      RCI_contrast=RCI)
    temp3 <- temp1 %>%
        dplyr::filter(Group==!!baseline) %>%
        dplyr::select(ModuleID, Group, RCI) %>%
        dplyr::rename(Group_baseline=Group,
                      RCI_baseline=RCI)
    temp <- dplyr::left_join(temp2, temp3, by="ModuleID") %>%
        dplyr::mutate(Comparison=!!comparison) %>%
        dplyr::select(Comparison, ModuleID,
                      Group_contrast, RCI_contrast, Group_baseline, RCI_baseline) %>%
        dplyr::bind_rows(temp, .)
}
print(nrow(temp))
head(temp)

#Simultaneously perform all tests using tidyr::nest()
temp <- temp %>%
    dplyr::group_by(Comparison) %>%
    tidyr::nest() %>%#New column name becomes "data"
    dplyr::mutate(Student=lapply(data, function(tbl) {
        t.test(tbl$RCI_contrast, tbl$RCI_baseline, alternative="two.sided", mu=0,
               paired=FALSE, var.equal=TRUE, conf.level=0.95)})) %>%
    dplyr::ungroup()
print(nrow(temp))
print(head(temp))#print() because Jupyter Lab tries to display list contents

model <- temp

In [ ]:
#Check result object
temp <- model$Student[[1]]
summary(temp)
for (name in rownames(summary(temp))) {
    print(name)
    print(temp[[name]])
    print("")
}

#### 3-2-2. Summarize all result objects into a table

In [ ]:
#Prepare summary table
temp <- tibble(ModuleID="All")
for (i in 1:length(contrast_vec)) {
    label <- contrast_vec[i]
    temp <- model %>%
        dplyr::filter(Comparison==!!label) %>%
        dplyr::mutate("{label}_DF":=sapply(Student, function(htest) {unname(htest[["parameter"]])}),
                      "{label}_Coef":=sapply(Student, function(htest) {
                          unname(htest[["estimate"]][1]-htest[["estimate"]][2])}),
                      "{label}_CoefSE":=sapply(Student, function(htest) {unname(htest[["stderr"]])}),
                      "{label}_tStat":=sapply(Student, function(htest) {unname(htest[["statistic"]])}),
                      "{label}_Pval":=sapply(Student, function(htest) {unname(htest[["p.value"]])}),
                      "{label}_AdjPval":=1.0) %>%#Insert dummy column for now
        dplyr::select(-Comparison, -data, -Student) %>%
        dplyr::bind_cols(temp, .)
}
##P-value adjustment across datasets with the Holm-Bonferroni method
temp1 <- temp %>%
    dplyr::select(ModuleID, all_of(str_c(contrast_vec,"_Pval"))) %>%
    tidyr::gather(key=ColName, value=Pval, -ModuleID) %>%
    dplyr::group_by(ModuleID) %>%
    dplyr::mutate(AdjPval=p.adjust(Pval, method="holm")) %>%
    dplyr::ungroup() %>%
    dplyr::select(-Pval) %>%
    dplyr::mutate(ColName=str_replace(ColName, "_Pval", "_AdjPval_temp")) %>%
    tidyr::spread(key=ColName, value=AdjPval)
##Replace the dummy values with the adjusted p-values
temp <- dplyr::left_join(temp, temp1, by="ModuleID")
for (i in 1:length(contrast_vec)) {
    label <- contrast_vec[i]
    temp <- temp %>%
        dplyr::mutate("{label}_AdjPval":=!!as.name(str_c(label,"_AdjPval_temp"))) %>%
        dplyr::select(-!!as.name(str_c(label,"_AdjPval_temp")))
}
print(str_c("nrow: ",nrow(temp)))
head(temp)

summary_tbl <- temp

In [ ]:
#Add general statistics
##Calculate general statistics
sem <- function(x) {sd(x)/sqrt(length(x))}
temp <- rci_kk %>%
    tidyr::gather(key=Group, value=RCI, -ModuleID) %>%
    dplyr::group_by(Group) %>%
    dplyr::summarize(Count=n(), RCImean=mean(RCI), RCIsem=sem(RCI))
temp1 <- tibble(ModuleID="All")
for (group in group_vec) {
    count <- temp$Count[which(temp$Group==group)]
    rcimean <- temp$RCImean[which(temp$Group==group)]
    rcisem <- temp$RCIsem[which(temp$Group==group)]
    temp1 <- temp1 %>%
        dplyr::mutate("{group}_N":=!!count,
                      "{group}_RCImean":=!!rcimean,
                      "{group}_RCIsem":=!!rcisem)
}
print(str_c("nrow: ",nrow(temp1)))
head(temp1)
##Merge
temp <- dplyr::left_join(temp1, summary_tbl, by="ModuleID")
print(str_c("nrow: ",nrow(temp)))
head(temp)

summary_tbl <- temp

In [ ]:
#Create a workbook object to save as one single .xlsx file
workbook <- createWorkbook()

#Prepare module metadata sheet
sheetName <- "ModuleMetadata"
addWorksheet(workbook, sheetName=sheetName)
writeData(workbook, sheetName, module_meta)

#Save the summary table as a new sheet
sheetName <- "RCImean"
addWorksheet(workbook, sheetName=sheetName)
writeData(workbook, sheetName, summary_tbl)

#Save the workbook as one single .xlsx file
fileDir <- "./ExportData/"
ipynbName <- "220529_LCproteomics-M004-DIRAC_StatisticalTest-GOBP-with-M001_ver2-3_"
fileName <- "inter-group-comparison.xlsx"
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

# — †1. Go back to †1 of the main Python notebook —  

## 4. Rank matching score under the M004 consensus: inter-group module comparison

> Test specific hypotheses: control RMS mean == intervention RMS mean per the M004-derived rank consensus for each module (i.e., inter-group module comparison).  
>
> 1. Testing the main effect of intervention on rank mathing scores (RMSs) for each module using ANOVA models  
> 2. Then, performing post-hoc comparisons of RMSs between control vs. each intervention using the repeated Student's t-tests  
>
> As well as using sex-pooled rank consensus, sex is NOT included in the ANOVA model. Instead of adding the rank consensus group (i.e., M004 group) and its interaction term to the ANOVA model, stratified ANOVA model is generated for each rank consensus group. Note that ANOVA can shrink the variance utilizing all samples (per module), whose statistical power is better than the repeated Welch's t-tests in the case of small sample size. Although tricky, the P-value adjustment in (1) is performed across all models (= modules x rank consensus groups) under the assumption that modules are independent, which would be more conservative and less likely raise referees' eyebrows for venn diagram-type summary than using nominal P-value cutoff. Because the post-hoc comparisons (2) are to address the similarity of each intervention within a specific module, the P-values are adjusted across interventions only within the module (not across modules). At the same time, these P-values are simultaneously adjusted across rank consensus groups within the module, because the hypotheses across rank consensus are independent in contrast to the previous M001-alone DIRAC analysis. Note that not Dunnett's test but Student's t-test (i.e., t-test with pooled variance) is used as the post-hoc test because further adjustment of the Dunnett's test p-values with the Holm-Bonferroni method is too much (incorrect) adjustment for family-wise error rate (FWER).

### 4-1. ANOVA test (RMS ~ Intervention), followed by repeated Student's t-tests (Intervention)

#### 4-1-1. Simultaneously perform all ANOVA tests

In [ ]:
#Prepare DF
template_vec <- c("Control (M004)", "4EGI-1")
temp <- rms_tbl %>%
    dplyr::filter(Template %in% template_vec) %>%
    tidyr::gather(key=SampleID, value=RMS, -ModuleID, -Template) %>%
    dplyr::left_join(., sample_meta, by="SampleID") %>%
    dplyr::filter(!(Phenotype %in% template_vec)) %>%#Eliminate the non-target groups
    dplyr::mutate(Intervention=str_replace(Intervention, "Estradiol", "17aE")) %>%#Exception
    dplyr::mutate(Intervention=str_c(Dataset,":",str_sub(Intervention, start=1, end=4))) %>%
    dplyr::mutate(Dataset=factor(Dataset, levels=c("M001", "M004")),
                  Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M")))
print(nrow(temp))
head(temp)

#Simultaneously perform all tests using tidyr::nest()
temp <- temp %>%
    dplyr::group_by(ModuleID, Template) %>%
    tidyr::nest() %>%#New column name becomes "data"
    dplyr::mutate(ANOVA=lapply(data, function(tbl) {aov(RMS~Intervention, data=tbl)})) %>%
    dplyr::ungroup()
print(nrow(temp))
print(head(temp))#print() because Jupyter Lab tries to display list contents

model1 <- temp

In [ ]:
#Check result object
summary(model1$ANOVA[[1]])

#### 4-1-2. Simultaneously perform all Student's t-tests

In [ ]:
#Prepare DF
temp1 <- rms_tbl %>%
    dplyr::filter(Template %in% template_vec) %>%
    tidyr::gather(key=SampleID, value=RMS, -ModuleID, -Template) %>%
    dplyr::left_join(., sample_meta, by="SampleID") %>%
    dplyr::mutate(Intervention=str_replace(Intervention, "Estradiol", "17aE")) %>%#Exception
    dplyr::mutate(Intervention=str_c(Dataset,":",str_sub(Intervention, start=1, end=4))) %>%
    dplyr::mutate(Dataset=factor(Dataset, levels=c("M001", "M004")),
                  Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M")))
comparison_vec <- c("M001:Acar-vs-M001:Cont", "M001:17aE-vs-M001:Cont", "M001:Rapa-vs-M001:Cont")
temp <- tibble()
for (comparison in comparison_vec) {
    contrast <- str_split(comparison, "-vs-", simplify=TRUE)[1]
    baseline <- str_split(comparison, "-vs-", simplify=TRUE)[2]
    temp2 <- temp1 %>%
        dplyr::filter(Intervention==!!contrast) %>%
        dplyr::select(ModuleID, Template, SampleID, RMS) %>%
        dplyr::rename(SampleID_contrast=SampleID,
                      RMS_contrast=RMS) %>%
        dplyr::group_by(ModuleID, Template) %>%
        dplyr::mutate(Sample_i=1:n()) %>%#Just for handling; no correspondence b/w baseline and contrast
        dplyr::ungroup()
    temp3 <- temp1 %>%
        dplyr::filter(Intervention==!!baseline) %>%
        dplyr::select(ModuleID, Template, SampleID, RMS) %>%
        dplyr::rename(SampleID_baseline=SampleID,
                      RMS_baseline=RMS) %>%
        dplyr::group_by(ModuleID, Template) %>%
        dplyr::mutate(Sample_i=1:n()) %>%#Just for handling; no correspondence b/w baseline and contrast
        dplyr::ungroup()
    temp <- dplyr::full_join(temp2, temp3, by=c("ModuleID", "Template", "Sample_i")) %>%
        dplyr::mutate(Comparison=!!comparison) %>%
        dplyr::select(ModuleID, Template, Comparison, Sample_i,
                      SampleID_contrast, RMS_contrast, SampleID_baseline, RMS_baseline) %>%
        dplyr::bind_rows(temp, .)
}
print(nrow(temp))
head(temp)

#Check NAs which can be derived by full_join when sample size is different b/w baseline and contrast
temp1 <- temp %>%
    dplyr::filter(is.na(RMS_contrast)|is.na(RMS_baseline)) %>%
    dplyr::group_by(ModuleID, Template, Comparison) %>%
    dplyr::summarize(Count=n()) %>%
    dplyr::ungroup()
print(str_c('Test with different sample size: ',nrow(temp1)))

#Simultaneously perform all tests using tidyr::nest()
temp <- temp %>%
    dplyr::group_by(ModuleID, Template, Comparison) %>%
    tidyr::nest() %>%#New column name becomes "data"
    dplyr::mutate(Student=lapply(data, function(tbl) {
        t.test(tbl$RMS_contrast, tbl$RMS_baseline, alternative="two.sided", mu=0,
               paired=FALSE, var.equal=TRUE, conf.level=0.95)})) %>%
    dplyr::ungroup()
print(nrow(temp))
print(head(temp))#print() because Jupyter Lab tries to display list contents

model2 <- temp

In [ ]:
#Check result object
temp <- model2$Student[[1]]
summary(temp)
for (name in rownames(summary(temp))) {
    print(name)
    print(temp[[name]])
    print("")
}

#### 4-1-3. Summarize all result objects into a table

In [ ]:
#Prepare variable labels
variable_vec <- rownames(summary(model1$ANOVA[[1]])[[1]]) %>%
    str_replace(., " *$", "")#Remove white spaces
variable_vec <- variable_vec[1:(length(variable_vec)-1)]#Remove Residuals

#Prepare summary table of ANOVA tests
temp1 <- model1 %>%
    dplyr::select(ModuleID, Template, ANOVA)
for (i in 1:length(variable_vec)) {
    label <- variable_vec[i]
    temp1 <- temp1 %>%
        dplyr::mutate("{label}_DF":=sapply(ANOVA, function(aov) {summary(aov)[[1]]$Df[i]}),
                      "{label}_SumSq":=sapply(ANOVA, function(aov) {summary(aov)[[1]]$`Sum Sq`[i]}),
                      "{label}_MeanSq":=sapply(ANOVA, function(aov) {summary(aov)[[1]]$`Mean Sq`[i]}),
                      "{label}_Fstat":=sapply(ANOVA, function(aov) {summary(aov)[[1]]$`F value`[i]}),
                      "{label}_Pval":=sapply(ANOVA, function(aov) {summary(aov)[[1]]$`Pr(>F)`[i]})) %>%
        #P-value adjustment with the Benjamini-Hochberg method
        ##Using !!as.name() in the following line, because simple {{}} and !! didn't recognize?
        dplyr::mutate("{label}_AdjPval":=p.adjust(!!as.name(str_c(label,"_Pval")), method="BH"))
}
temp1 <- temp1 %>%
    dplyr::select(-ANOVA)
print(str_c("nrow: ",nrow(temp1)))
head(temp1)

#Prepare summary table of t-tests
temp2 <- model2 %>%
    dplyr::select(ModuleID, Template) %>%
    dplyr::distinct()
for (i in 1:length(comparison_vec)) {
    label <- comparison_vec[i]
    temp2 <- model2 %>%
        dplyr::filter(Comparison==!!label) %>%
        dplyr::mutate("{label}_DF":=sapply(Student, function(htest) {unname(htest[["parameter"]])}),
                      "{label}_Coef":=sapply(Student, function(htest) {
                          unname(htest[["estimate"]][1]-htest[["estimate"]][2])}),
                      "{label}_CoefSE":=sapply(Student, function(htest) {unname(htest[["stderr"]])}),
                      "{label}_tStat":=sapply(Student, function(htest) {unname(htest[["statistic"]])}),
                      "{label}_Pval":=sapply(Student, function(htest) {unname(htest[["p.value"]])}),
                      "{label}_AdjPval":=1.0) %>%#Insert dummy column for now
        dplyr::select(-Comparison, -data, -Student) %>%
        dplyr::left_join(temp2, ., by=c("ModuleID", "Template"))
}
##P-value adjustment across interventions and templates with the Holm-Bonferroni method per module
temp3 <- temp2 %>%
    dplyr::select(ModuleID, Template, all_of(str_c(comparison_vec,"_Pval"))) %>%
    tidyr::gather(key=ColName, value=Pval, -ModuleID, -Template) %>%
    dplyr::group_by(ModuleID) %>%
    dplyr::mutate(AdjPval=p.adjust(Pval, method="holm")) %>%
    dplyr::ungroup() %>%
    dplyr::select(-Pval) %>%
    dplyr::mutate(ColName=str_replace(ColName, "_Pval", "_AdjPval_temp")) %>%
    tidyr::spread(key=ColName, value=AdjPval)
##Replace the dummy values with the adjusted p-values
temp2 <- dplyr::left_join(temp2, temp3, by=c("ModuleID", "Template"))
for (i in 1:length(comparison_vec)) {
    label <- comparison_vec[i]
    temp2 <- temp2 %>%
        dplyr::mutate("{label}_AdjPval":=!!as.name(str_c(label,"_AdjPval_temp"))) %>%
        dplyr::select(-!!as.name(str_c(label,"_AdjPval_temp")))
}
print(str_c("nrow: ",nrow(temp2)))
head(temp2)

#Merge
temp <- dplyr::left_join(temp1, temp2, by=c("ModuleID", "Template"))
print(str_c("nrow: ",nrow(temp)))
head(temp)

summary_tbl <- temp

In [ ]:
#Add general statistics
##Calculate general statistics
sem <- function(x) {sd(x)/sqrt(length(x))}
temp <- rms_tbl %>%
    dplyr::filter(Template %in% template_vec) %>%
    tidyr::gather(key=SampleID, value=RMS, -ModuleID, -Template) %>%
    dplyr::left_join(., sample_meta, by="SampleID") %>%
    dplyr::mutate(Intervention=str_replace(Intervention, "Estradiol", "17aE")) %>%#Exception
    dplyr::mutate(Intervention=str_c(Dataset,":",str_sub(Intervention, start=1, end=4))) %>%
    dplyr::mutate(Dataset=factor(Dataset, levels=c("M001", "M004")),
                  Intervention=factor(Intervention, levels=group_vec),
                  Sex=factor(Sex, levels=c("F", "M"))) %>%
    dplyr::group_by(ModuleID, Template, Intervention) %>%
    dplyr::summarize(Count=n(), RMSmean=mean(RMS), RMSsem=sem(RMS)) %>%
    dplyr::ungroup()
temp1 <- module_meta %>%
    dplyr::select(ModuleID, ModuleName)
temp1 <- dplyr::bind_rows(temp1, temp1) %>%
    dplyr::mutate(Template=rep(template_vec, each=nrow(module_meta)))
for (group in group_vec) {
    temp1 <- temp %>%
        dplyr::filter(Intervention==!!group) %>%
        dplyr::select(-Intervention) %>%
        dplyr::rename("{group}_N":=Count,
                      "{group}_RMSmean":=RMSmean,
                      "{group}_RMSsem":=RMSsem) %>%
        dplyr::left_join(temp1, ., by=c("ModuleID", "Template"))
}
print(str_c("nrow: ",nrow(temp1)))
head(temp1)
temp <- dplyr::left_join(temp1, summary_tbl, by=c("ModuleID", "Template")) %>%
    dplyr::arrange(Intervention_Pval)
print(str_c("nrow: ",nrow(temp)))
head(temp)

summary_tbl <- temp

> (Note that the beta-coefficient estimate is equivalent to the difference in the mean of RMSs; e.g., Acar-vs-Cont_Coef = Acar_RMSmean - Cont_RMSmean.)  

In [ ]:
#Save by appending to the existing .xlsx file
##Load the existing .xlsx file as a new workbook object
fileDir <- "./ExportData/"
ipynbName <- "220529_LCproteomics-M004-DIRAC_StatisticalTest-GOBP-with-M001_ver2-3_"
fileName <- "inter-group-comparison.xlsx"
workbook <- loadWorkbook(str_c(fileDir,ipynbName,fileName))
##Add the tibble object as a new sheet per template
for (template in template_vec) {
    temp <- summary_tbl %>%
        dplyr::filter(Template==!!template) %>%
        dplyr::select(-Template)
    print(str_c("nrow: ",nrow(temp)))
    print(head(temp))
    
    if (template=="Control (M004)") {
        sheetName <- "M004-Cont-fixed-RMSmean"#Colon not allowed in sheet name in Excel...
    } else if (template=="4EGI-1") {
        sheetName <- "M004-4EGI-fixed-RMSmean"#Colon not allowed in sheet name in Excel...
    } else {
        sheetName <- "Error?"
    }
    addWorksheet(workbook, sheetName=sheetName)
    writeData(workbook, sheetName, temp)
}
##Update the existing .xlsx file
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

# — †2. Go back to †2 of the main Python notebook —  

# — Session information —

In [ ]:
sessionInfo()